In [1]:
from keras.models import load_model
import cv2
import numpy as np
import os

In [2]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
print("Loading model...")
model = load_model("Keras model/keras_model.h5", compile=False)
print("Model loaded successfully.")

# Load the labels
print("Loading labels...")
class_names = open("Keras model/labels.txt", "r").readlines()
# Remove leading/trailing whitespace
class_names = [cls.strip() for cls in class_names]
print("Labels loaded successfully.")

# Directory containing images
image_dir = "Face_Output/Face_Cropped_Split"

# Output directory for storing confidence scores
output_dir = "Face_Output/Face_Label_Scores"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get list of subdirectories (Train and Test)
subdirectories = ["Train", "Test"]

# Variables for calculating accuracy
total_images = 0
correct_predictions = 0

for subdir in subdirectories:
    print(f"Processing images in {subdir} directory...")
    subdir_path = os.path.join(image_dir, subdir)
    # Get list of image filenames in the subdirectory
    image_filenames = os.listdir(subdir_path)

    for filename in image_filenames:
        print(f"Processing image: {filename}")
        # Load image
        image_path = os.path.join(subdir_path, filename)
        image = cv2.imread(image_path)

        # Resize the image to (224, 224) pixels
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

        # Make the image a numpy array and reshape it to the model's input shape
        image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

        # Normalize the image array
        image = (image / 127.5) - 1

        # Predicts the model
        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Get the ground truth label from the filename
        ground_truth_label = filename.split("_")[0]

        # Compare predicted class with ground truth label
        if class_name == ground_truth_label:
            correct_predictions += 1
            prediction_result = "Correct"
        else:
            prediction_result = "Incorrect"

        total_images += 1

        # Print prediction and confidence score
        print("Prediction:")
        print("  Image:", filename)
        print("  Class:", class_name)
        print("  Confidence Score:", str(
            np.round(confidence_score * 100))[:-2], "%")
        print("  Prediction Result:", prediction_result)
        print("  Ground Truth Label:", ground_truth_label)
        print("----------------------")

        # Store confidence score in an array
        confidence_scores_array = []

        # Append confidence score to array
        confidence_scores_array.append({
            "filename": filename,
            "class_name": class_name,
            "confidence_score": confidence_score,
            "prediction_result": prediction_result,
            "ground_truth_label": ground_truth_label
        })

    # Store confidence scores array to a file (Optional)
    np.save(os.path.join(
        output_dir, f"{subdir}_confidence_scores.npy"), confidence_scores_array)

# Calculate accuracy
accuracy = (correct_predictions / total_images) * 100
print("Overall Accuracy:", accuracy, "%")

print("All images processed successfully.")

Loading model...


Model loaded successfully.
Loading labels...
Labels loaded successfully.
Processing images in Train directory...
Processing image: S0001_F_30_10_Cropped.jpg
1/1 [==============================] - 1s 981ms/step
Prediction:
  Image: S0001_F_30_10_Cropped.jpg
  Class: 17 S0018_F_43
  Confidence Score: 94 %
  Prediction Result: Incorrect
  Ground Truth Label: S0001
----------------------
Processing image: S0001_F_30_11_Cropped.jpg
1/1 [==============================] - 0s 33ms/step
Prediction:
  Image: S0001_F_30_11_Cropped.jpg
  Class: 9 S0010_M_24
  Confidence Score: 54 %
  Prediction Result: Incorrect
  Ground Truth Label: S0001
----------------------
Processing image: S0001_F_30_14_Cropped.jpg
1/1 [==============================] - 0s 33ms/step
Prediction:
  Image: S0001_F_30_14_Cropped.jpg
  Class: 17 S0018_F_43
  Confidence Score: 79 %
  Prediction Result: Incorrect
  Ground Truth Label: S0001
----------------------
Processing image: S0001_F_30_17_Cropped.jpg
1/1 [